# Introduction
For my final capstone, I will create a model that processes a piece of an xray of breast tissue and classifies whether or not image contains cancerous tissue. Breast cancer is the most common form of cancer in women, contributing just over a quarter of diagnoses, so having the ability to improve the timeline and accuracy of a diagnosis would greatly improve the disease.

My solution will be valuable because it will capture this information more quickly and accurately than a person could. It would be a model used by hospitals and other healthcare organizations to identify the presence of cancerous tissue. This model would be able to see abnormalities at a much earlier stage which would allow treatment to commence earlier and have a higher success rate.  Ideally, the model could be adjusted for other types of tumor or disease classifications.

I will to use the [Breast Histopathology Images dataset](https://www.kaggle.com/paultimothymooney/breast-histopathology-images) from the National Library of Medicine. There are 277,524 pieces of 162 images scanned which should be plenty of data to learn from. The data is imbalanced - there is a 34/66 split between the classes. To handle this, I will likely choose a random subset of the images marked IDC negative to ensure I have a balanced dataset.

Because the data is images, I don't believe it doesn't to be cleaned and prepped - it will simply be imported into Python and converted into a dataframe in which each column represents a pixel. I will then use dimensionality reduction and unsupervised learning to obtain in-depth knowledge about the information the data provides and visualize any patterns or clusters the information may fall into. I will need to do some research on how to use supervised learning on image processing/classification as it is not yet clear to me how to do so. Finally, I will use neural networks, and CNN in particular, to classify the scans and create a model that can take an un-labeled scan and report whether or not cancerous tissue is present. 

I anticipate importing and processing the images as well as tuning the parameters in the neural networks to be the biggest challenges I'll face. I plan on spending the majority of my time on these two items. For the most part, I should be able to recycle code from previous projects which should expedite my prepping and modeling. I'll do additional research on how best to use neural networks for image classification in an effort to obtain the best model.


In [2]:
import os
os.chdir('C:\\Users\\M246047\\Documents\\Python')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta
import re
import tensorflow as tf
import keras
from zipfile import ZipFile
import cv2
import glob

# Preprocessing/Cleaning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import scipy.stats as stats

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Dimensionality Reduction & Unsupervised Learning
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, TSNE
import umap
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
import hdbscan
from sklearn import datasets, metrics
from sklearn.neural_network import BernoulliRBM
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage as lnkg
from sklearn.neural_network import MLPClassifier


# Import the backend
from keras import backend as K
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


# Importing and Prepping the Data

I gravely underestimated how difficult this task would be. I had an initial folder for the data which contained folders for each patient. Each of these folders contained two folders labeled 0 and 1 which contained images with or without cancerous tissue, depending on the folder. At first, I tried to iterate through each patient folder to grab the 0 and 1 folders within, and iterate through each of those folder to grab each image to import it, but was unable to do so. In the end I mapped to the initial folder and iterated through each patient folder to create arrays containing the pixel data for each image. Using the length of these arrays, I was able to create an array with the target values, with 0 for a scan without cancerous tissue and 1 for a scan with.



# Playing with Importing

In [111]:
image = cv2.imread('8863_idx5_x51_y1251_class0.png', 0)
print(type(image))
image.shape

<class 'numpy.ndarray'>


(50, 50)

In [97]:
x = image.reshape((1,) + image.shape)
print(x.shape)
x

(1, 50, 50)


array([[[247, 246, 247, ..., 131, 109, 117],
        [246, 246, 246, ..., 136, 130, 139],
        [246, 246, 246, ..., 133, 125, 113],
        ...,
        [246, 246, 246, ..., 241, 243, 246],
        [246, 246, 246, ..., 246, 244, 244],
        [246, 246, 245, ..., 242, 245, 244]]], dtype=uint8)

In [118]:
image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8864\\0\\8864_idx5_x1001_y1501_class0.png', 0)
print(image_data)

[[219 232 241 ... 235 242 242]
 [217 238 242 ... 241 238 237]
 [227 235 241 ... 224 217 234]
 ...
 [239 239 240 ... 241 241 240]
 [240 241 241 ... 237 241 240]
 [240 240 239 ... 233 241 241]]


# Importing the Images by Folder

In [3]:
images_0_8863 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8863\\0'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8863\\0\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data = image_data.reshape(1, 50, 50)
            images_0_8863.append(image_data)
            
images_0_8863 = np.asarray(images_0_8863)    
print('There are {} images'.format(len(images_0_8863)))
print('The type of the data is', type(images_0_8863))
print('The shape of the data is', images_0_8863.shape)
images_0_8863_class = np.zeros((len(images_0_8863),), dtype=int)

There are 768 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (768, 50, 50)


In [4]:
# Creating a list of image names
images_1_8863 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8863\\1'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8863\\1\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data = image_data.reshape(2500,)
            images_1_8863.append(image_data)
            
images_1_8863 = np.asarray(images_1_8863)
print('There are {} images'.format(len(images_1_8863)))
print('The type of the data is', type(images_1_8863))
print('The shape of the data is', images_1_8863.shape)
images_1_8863_class = np.ones((len(images_1_8863),), dtype=int)

There are 207 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (207, 50, 50)


In [5]:
images_0_8864 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8864\\0'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8864\\0\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data = image_data.reshape(2500,)
            images_0_8864.append(image_data)
            
images_0_8864 = np.asarray(images_0_8864)    
print('There are {} images'.format(len(images_0_8864)))
print('The type of the data is', type(images_0_8864))
print('The shape of the data is', images_0_8864.shape)
images_0_8864_class = np.zeros((len(images_0_8864)), dtype=int)

There are 730 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (730, 50, 50)


In [6]:
# Creating a list of image names
images_1_8864 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8864\\1'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8864\\1\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data = image_data.reshape(2500,)
            images_1_8864.append(image_data)
            
images_1_8864 = np.asarray(images_1_8864)
print('There are {} images'.format(len(images_1_8864)))
print('The type of the data is', type(images_1_8864))
print('The shape of the data is', images_1_8864.shape)
images_1_8864_class = np.ones((len(images_1_8864),), dtype=int)

There are 326 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (326, 50, 50)


In [7]:
images_0_8865 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8865\\0'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8865\\0\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data = image_data.reshape(2500,)
            images_0_8865.append(image_data)
            
images_0_8865 = np.asarray(images_0_8865)    
print('There are {} images'.format(len(images_0_8865)))
print('The type of the data is', type(images_0_8865))
print('The shape of the data is', images_0_8865.shape)
images_0_8865_class = np.zeros((len(images_0_8865)), dtype=int)

There are 657 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (657, 50, 50)


In [8]:
# Creating a list of image names
images_1_8865 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8865\\1'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8865\\1\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data = image_data.reshape(2500,)
            images_1_8865.append(image_data)
            
images_1_8865 = np.asarray(images_1_8865)
print('There are {} images'.format(len(images_1_8865)))
print('The type of the data is', type(images_1_8865))
print('The shape of the data is', images_1_8865.shape)
images_1_8865_class = np.ones((len(images_1_8865),), dtype=int)

There are 55 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (55, 50, 50)


# Defining/Cleaning/Normalizing the Data/Images/Pixels

In [8]:
# Create x_train and y_train
print(images_0_8863.shape)
print(images_1_8863.shape)
print(images_0_8863_class.shape)
print(images_1_8863_class.shape)


x_train = np.append(images_0_8863, images_1_8863, axis=0)
y_train = np.append(images_0_8863_class, images_1_8863_class, axis=0)
print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)

# Create x_test and y_test
x_test = np.append(images_0_8865, images_1_8865, axis=0)
y_test = np.append(images_0_8865_class, images_1_8865_class, axis=0)

print('x_test shape:', x_test.shape)
print('y_test shape: ', y_test.shape)

(768, 50, 50)
(207, 50, 50)
(768,)
(207,)
x_train shape: (975, 50, 50)
y_train shape: (975,)
x_test shape: (712, 50, 50)
y_test shape:  (712,)


In [9]:
# Create x_train and y_train
print(images_0_8863.shape)
print(images_1_8863.shape)
print(images_0_8863_class.shape)
print(images_1_8863_class.shape)
print(images_0_8865.shape)
print(images_1_8865.shape)
print(images_0_8865_class.shape)
print(images_1_8865_class.shape)

x_train = np.append(images_0_8863, images_1_8863, axis=0)
x1_train = np.append(x_train, images_0_8865, axis=0)
x2_train = np.concatenate((images_0_8863, images_1_8863,images_1_8865), axis=0)
y_train = np.append(images_0_8863_class, images_1_8863_class, axis=0)
print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)

# Create x_test and y_test
x_test = np.append(images_0_8865, images_1_8865, axis=0)
y_test = np.append(images_0_8865_class, images_1_8865_class, axis=0)

print('x_test shape:', x_test.shape)
print('y_test shape: ', y_test.shape)

(768, 50, 50)
(207, 50, 50)
(768,)
(207,)
(657, 50, 50)
(55, 50, 50)
(657,)
(55,)
x_train shape: (975, 50, 50)
y_train shape: (975,)
x_test shape: (712, 50, 50)
y_test shape:  (712,)


In [10]:
# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

x_train shape: (975, 50, 50)
x_test shape: (712, 50, 50)


In [11]:
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (975, 50, 50)
y_train shape:  (975, 2)
x_test shape:  (712, 50, 50)
y_test shape:  (712, 2)


In [12]:
# Per Yunus: first of all keep your images as (1,50,50) not as (1,2500).
img_rows = 50
img_cols = 50
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [13]:
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (975, 50, 50, 1)
y_train shape:  (975, 2)
x_test shape:  (712, 50, 50, 1)
y_test shape:  (712, 2)


# Creating the Neural Networks

## Model 1: Simple Sequential

In [14]:
# Start with a simple sequential model
model = Sequential()
model.add(Flatten())
# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu'))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(2, activation='softmax'))



# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [17]:

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
model.summary()
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 975 samples, validate on 712 samples
Epoch 1/10
975/975 [==============================] - 0s 252us/step - loss: 1.9365 - accuracy: 0.6431 - val_loss: 0.2549 - val_accuracy: 0.9228
Epoch 2/10
975/975 [==============================] - 0s 83us/step - loss: 0.7903 - accuracy: 0.6995 - val_loss: 0.3161 - val_accuracy: 0.9228
Epoch 3/10
975/975 [==============================] - 0s 76us/step - loss: 0.5784 - accuracy: 0.7497 - val_loss: 0.2403 - val_accuracy: 0.9228
Epoch 4/10
975/975 [==============================] - 0s 58us/step - loss: 0.5075 - accuracy: 0.7590 - val_loss: 0.2498 - val_accuracy: 0.9228
Epoch 5/10
975/975 [==============================] - 0s 60us/step - loss: 0.5150 - accuracy: 0.7723 - val_loss: 0.2527 - val_accuracy: 0.9228
Epoch 6/10
975/975 [==============================] - 0s 62us/step - loss: 0.4671 - accuracy: 0.7856 - val_loss: 0.2556 - val_accuracy: 0.9228
Epoch 7/10
975/975 [==============================] - 0s 76us/step - loss: 0.5074 - accuracy: 0

## Model 2: Convolutional Neural Network

In [19]:
# Building the Model
num_classes = 2
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 975 samples, validate on 712 samples
Epoch 1/10
975/975 [==============================] - 6s 6ms/step - loss: 0.7329 - accuracy: 0.7528 - val_loss: 0.2607 - val_accuracy: 0.9228
Epoch 2/10
975/975 [==============================] - 6s 6ms/step - loss: 0.4748 - accuracy: 0.7877 - val_loss: 0.2544 - val_accuracy: 0.9228
Epoch 3/10
975/975 [==============================] - 6s 6ms/step - loss: 0.4687 - accuracy: 0.7877 - val_loss: 0.4076 - val_accuracy: 0.9228
Epoch 4/10
975/975 [==============================] - 7s 7ms/step - loss: 0.4644 - accuracy: 0.7877 - val_loss: 0.2352 - val_accuracy: 0.9228
Epoch 5/10
975/975 [==============================] - 7s 7ms/step - loss: 0.4681 - accuracy: 0.7877 - val_loss: 0.3682 - val_accuracy: 0.9228
Epoch 6/10
975/975 [==============================] - 6s 6ms/step - loss: 0.4401 - accuracy: 0.7877 - val_loss: 0.2260 - val_accuracy: 0.9228
Epoch 7/10
975/975 [==============================] - 7s 7ms/step - loss: 0.4630 - accuracy: 0.7979 - 

## Model 3: Hierarchical Recurrrent Neural Networks

In [119]:
# Training parameters.
batch_size = 64
num_classes = 2
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32


row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 975 samples, validate on 712 samples
Epoch 1/3
975/975 [==============================] - 10s 10ms/step - loss: 0.5131 - accuracy: 0.7559 - val_loss: 0.2811 - val_accuracy: 0.9228
Epoch 2/3
975/975 [==============================] - 11s 11ms/step - loss: 0.4294 - accuracy: 0.7877 - val_loss: 0.2883 - val_accuracy: 0.9228
Epoch 3/3
975/975 [==============================] - 12s 12ms/step - loss: 0.4143 - accuracy: 0.7877 - val_loss: 0.2642 - val_accuracy: 0.9228
Test loss: 0.2642165682791324
Test accuracy: 0.9227527976036072


In [121]:
# Create x_train and y_train
print(images_0_8863.shape)
print(images_1_8863.shape)

x_train = np.append(images_0_8863, [images_1_8863], axis=0)
y_train = np.append(images_0_8863_class, [images_1_8863_class, images_0_8864_class, images_1_8864_class], axis=0)

# Create x_test and y_test
x_test = np.append(images_0_8865, images_1_8865, axis=0)
y_test = np.append(images_0_8865_class, images_1_8865_class, axis=0)

(768, 50, 50)
(207, 50, 50)


ValueError: all the input arrays must have same number of dimensions

In [268]:
x_train.shape

(975, 2500)

In [287]:
# Create x_train and y_train
print(images_0_8863.shape)
print(images_1_8863.shape)
print(images_0_8863_class.shape)
print(images_1_8863_class.shape)

x_train = np.append(images_0_8863, images_1_8863, axis=0)
y_train = np.append(images_0_8863_class, images_1_8863_class, axis=0)
print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)

# Create x_test and y_test
x_test = np.append(images_0_8865, images_1_8865, axis=0)
y_test = np.append(images_0_8865_class, images_1_8865_class, axis=0)

print('x_test shape:', x_test.shape)
print('y_test shape: ', y_test.shape)

(768, 2500)
(207, 2500)
(768,)
(207,)
x_train shape: (975, 2500)
y_train shape: (975,)
x_test shape: (712, 2500)
y_test shape:  (712,)


In [288]:
# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255
print('x_train shape', x_train.shape)
print('x_test shape', x_test.shape)
# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

x_train shape (975, 2500)
x_test shape (712, 2500)


In [289]:
x_train
y_test.shape

(712, 2)

I think the arrays need to be (X, 2500) <br>
Where is the standardization (/255) coming from?

In [290]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(2500,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                160064    
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 164,874
Trainable params: 164,874
Non-trainable params: 0
_________________________________________________________________


In [115]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 975 samples, validate on 712 samples
Epoch 1/10
975/975 [==============================] - 0s 98us/step - loss: 0.4301 - accuracy: 0.7836 - val_loss: 0.2462 - val_accuracy: 0.9228
Epoch 2/10
975/975 [==============================] - 0s 50us/step - loss: 0.4341 - accuracy: 0.7826 - val_loss: 0.2984 - val_accuracy: 0.9228
Epoch 3/10
975/975 [==============================] - 0s 52us/step - loss: 0.4212 - accuracy: 0.7877 - val_loss: 0.2419 - val_accuracy: 0.9228
Epoch 4/10
975/975 [==============================] - 0s 46us/step - loss: 0.4456 - accuracy: 0.7805 - val_loss: 0.2353 - val_accuracy: 0.9228
Epoch 5/10
975/975 [==============================] - 0s 56us/step - loss: 0.4205 - accuracy: 0.7846 - val_loss: 0.2560 - val_accuracy: 0.9228
Epoch 6/10
975/975 [==============================] - 0s 52us/step - loss: 0.4204 - accuracy: 0.7887 - val_loss: 0.2676 - val_accuracy: 0.9228
Epoch 7/10
975/975 [==============================] - 0s 56us/step - loss: 0.4273 - accuracy: 0.

In [51]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(975,50,50,)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


In [53]:
print('x test shape: ', x_test.shape)
print('y test shape: ', y_test.shape)

x test shape:  (712, 50, 50, 1)
y test shape:  (712, 2)


In [52]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Error when checking input: expected conv2d_9_input to have shape (975, 50, 50) but got array with shape (50, 50, 1)

In [216]:
X_data = []
files = glob.glob("*.png")
for myFile in files:
    print(myFile)
    image = cv2.imread(myFile, 0)
    X_data.append(image)

print('X_data shape:', np.array(X_data).shape)

8863_idx5_x51_y1251_class0.png
X_data shape: (1, 50, 50)


In [ ]:
Loop over directory structure
within each subdirectory, read the files as gray scale
read the array, reshape


In [9]:
# The data

# Create x_train and y_train
print(images_0_8863.shape)
print(images_1_8863.shape)
print(images_0_8863_class.shape)
print(images_1_8863_class.shape)
print(images_0_8864.shape)
print(images_1_8864.shape)
print(images_0_8864_class.shape)
print(images_1_8864_class.shape)
print(images_0_8865.shape)
print(images_1_8865.shape)
print(images_0_8865_class.shape)
print(images_1_8865_class.shape)

images = [images_0_8863, images_1_8863, images_0_8863_class, images_1_8863_class, images_0_8864, images_1_8864,
          images_0_8864_class, images_1_8864_class, images_0_8865, images_1_8865, images_0_8865_class, images_1_8865_class]

# for image in images: 
#     print(image.shape)
#     image = image.reshape(1, 2500)

X = np.concatenate((images_0_8863, images_1_8863,images_0_8864, images_1_8864, images_0_8865, images_1_8865), axis=0)
X = X.reshape(2743, 2500)/255.0
y = np.concatenate((images_0_8863_class, images_1_8863_class,images_0_8864_class, images_1_8864_class, images_0_8865_class,
                                         images_1_8865_class), axis=0)
print('X shape:', X.shape)
print('y shape:', y.shape)


(768, 50, 50)
(207, 50, 50)
(768,)
(207,)
(730, 50, 50)
(326, 50, 50)
(730,)
(326,)
(657, 50, 50)
(55, 50, 50)
(657,)
(55,)
X shape: (2743, 2500)
y shape: (2743,)


# Dimensionality Reduction

## PCA

In [28]:
# We just want the first two principal components
pca = PCA(n_components=2)

# We get the components by 
# calling fit_transform method with our data
pca_components = pca.fit_transform(X)
print(pca_components)

[[ 9.45088837  0.75192309]
 [14.13882234  0.74325024]
 [ 6.8633756  -0.66982724]
 ...
 [ 4.37892026  0.8898027 ]
 [-0.37017609  0.9164232 ]
 [-2.37185332  1.74217311]]


In [29]:
plt.figure(figsize=(10,5))
colours = ["r","b"]
for i in range(pca_components.shape[0]):
    plt.text(pca_components[i, 0], pca_components[i, 1], str(y[i]),
             color=colours[int(y[i])],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(pca_components[:, 0], pca_components[:, 1])
plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

## t-SNE

In [12]:
for perplexity in [10, 20, 30, 40]:
    tsne = TSNE(n_components=2, verbose=1, perplexity=perplexity, n_iter=300)
    tsne_results = tsne.fit_transform(X)
    print('Cluster Graph with t-SNE with Perplexity ', perplexity)

#     plt.figure(figsize=(8,3))
#     plt.scatter(tsne_results[:, 0], tsne_results[:, 1])
#     plt.xticks([])
#     plt.yticks([])
#     plt.axis('off')
#     plt.show()
    
    plt.figure(figsize=(10,5))
    colours = ["r","b"]
    for i in range(tsne_results.shape[0]):
        plt.text(tsne_results[i, 0], tsne_results[i, 1], str(y[i]),
             color=colours[int(y[i])],
             fontdict={'weight': 'bold', 'size': 5}
        )

    plt.xticks([])
    plt.yticks([])
    plt.axis('off')
    plt.show()

[t-SNE] Computing 31 nearest neighbors...
[t-SNE] Indexed 2743 samples in 0.475s...
[t-SNE] Computed neighbors for 2743 samples in 66.683s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2743
[t-SNE] Computed conditional probabilities for sample 2000 / 2743
[t-SNE] Computed conditional probabilities for sample 2743 / 2743
[t-SNE] Mean sigma: 1.505131
[t-SNE] KL divergence after 250 iterations with early exaggeration: 124.958084
[t-SNE] KL divergence after 300 iterations: 4.538259
Cluster Graph with t-SNE with Perplexity  10


ValueError: Image size of 1255441x277357 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 720x360 with 1 Axes>

[t-SNE] Computing 61 nearest neighbors...
[t-SNE] Indexed 2743 samples in 0.429s...


KeyboardInterrupt: 

In [13]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)
plt.figure(figsize=(10,5))
colours = ["r","b","g","c","m","y","k","r","burlywood","chartreuse"]
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(y[i]),
             color=colours[int(y[i])],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 2743 samples in 0.503s...
[t-SNE] Computed neighbors for 2743 samples in 64.672s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2743
[t-SNE] Computed conditional probabilities for sample 2000 / 2743
[t-SNE] Computed conditional probabilities for sample 2743 / 2743
[t-SNE] Mean sigma: 2.006664
[t-SNE] KL divergence after 250 iterations with early exaggeration: 91.608643
[t-SNE] KL divergence after 300 iterations: 2.627568


ValueError: Image size of 176057x36068 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 720x360 with 1 Axes>

## UMAP

In [19]:
import umap.UMAP as umap

ModuleNotFoundError: No module named 'umap.UMAP'

In [15]:
# UMAP with all features
print('UMAP with all features. \n')
for n_neighbors in [5, 10]:
    for min_dist in [0.1, 0.25]:
        umap_results = umap.UMAP(n_neighbors=n_neighbors,
                      min_dist=min_dist,
                      metric='correlation').fit_transform(X)
        print('UMAP for X_all with ', n_neighbors, 'neighbors and ', min_dist, 'minimum distance.')
        plt.figure(figsize=(10,5))
        plt.scatter(umap_results[:, 0], umap_results[:, 1])
        plt.xticks([])
        plt.yticks([])
        plt.axis('off')
        plt.show()
        print('\n')

UMAP with all features. 



AttributeError: module 'umap' has no attribute 'umap'

## Functions to Add Cluster Labels

For each clustering technique, I'll create a function to run it. I'll use lists and dictionaries to compare parameters (cluster numbers, eps, etc.), so I've also created a function (getKeysByValue) that will allow me to obtain a dictionary key by its value. Additionally, I've created two functions that will obtain the p-values for all variables once clusters have been assigned. One will be used when a clustering technique provides two clusters, and the other for three.

In [20]:

# Get a list of keys from dictionary which has the given value

def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys

import operator

def get_pvalue_2clusters(variables):
    variables_pvalues = {}
    for variable in variables:
        t, pvalue = stats.ttest_ind(group_0[[variable]], group_1[[variable]])
        variables_pvalues[variable] = pvalue
    sorted_variables_pvalues = sorted(variables_pvalues.items(), key=operator.itemgetter(1))
    for variable, pvalue in sorted_variables_pvalues:
        print(variable, ':', pvalue)

def get_pvalue_3clusters(variables):
    variables_pvalues = {}
    for variable in variables:
        stat, pvalue = stats.f_oneway(group_0[[variable]], group_1[[variable]], group_2[[variable]])
        variables_pvalues[variable] = pvalue
    sorted_variables_pvalues = sorted(variables_pvalues.items(), key=operator.itemgetter(1))
    for variable, pvalue in sorted_variables_pvalues:
        print(variable, ':', pvalue)

# Clustering
## K-means Function
All functions will be able to use data with either PCA or UMAP components and additional parameters when needed. They will iterate through a set number of clusters (or other parameter), graph the silhouette scores, and plot the clusters when able. I have also chosen to show the Davies Bouldin index as another clustering metric to compare.

In [22]:
def kmeans_clustering(df, dr_technique, **params):    
    clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    silhouettes_list = []
    silhouettes_dict = {} # silhoutte scores are keys, clusters are values
    dbs = []      
       
   
    if dr_technique=='PCA':
        pca = PCA(n_components=2)
        components = pca.fit_transform(df)
       
    elif dr_technique=='UMAP': 
        parameters = list(params.keys())
        number_of_neighbors = params[parameters[0]]
        min_distance = params[parameters[1]]
        components = umap.UMAP(n_neighbors=number_of_neighbors, min_dist=min_distance, metric='correlation').fit_transform(df)

    # looping through cluster numbers to find the best cluster
    for cluster in clusters:
        labels = KMeans(n_clusters=cluster, random_state=123).fit_predict(components)
        kmeans_cluster = KMeans(n_clusters=cluster, random_state=123)
        y_pred = kmeans_cluster.fit_predict(components)
        silhouette = metrics.silhouette_score(components, labels, metric='euclidean')
        silhouettes_list.append(silhouette)
        silhouettes_dict[cluster] = silhouette
        db = metrics.davies_bouldin_score(components, labels)
        dbs.append(db)

    # plotting the silhouette scores
    plt.title('Silhouette Scores for Number of Clusters')  
    plt.scatter(list(silhouettes_dict.keys()), list(silhouettes_dict.values()))
    plt.show()


    best_cluster = getKeysByValue(silhouettes_dict, max(silhouettes_list))
    best_cluster_dbi = metrics.davies_bouldin_score(components,  KMeans(n_clusters=best_cluster[0], random_state=123).fit_predict(components))
    print('Best Silhouette score is {} with {} clusters. '.format( silhouettes_dict[best_cluster[0]],best_cluster[0]))
    print('Best Davies Bouldin score: ', min(dbs))   
    # plotting the clusters with the best silhouette score
    labels = KMeans(n_clusters=best_cluster[0], random_state=123).fit_predict(components)
    kmeans_cluster = KMeans(n_clusters=best_cluster[0], random_state=123)
    y_pred = kmeans_cluster.fit_predict(components)
       
    plt.title('K-Means Clustering for {} Clusters with Davies Bouldin Index {}'.format(best_cluster[0], best_cluster_dbi))
    plt.scatter(components[:,0], components[:,1], c=y_pred)
    plt.show()

## P-Values for Labeled Features
For each round of clustering, I will assign cluster labels to the highest performer, find the p-values between clusters, and graph the results. In this round, using PCA components for the features highly correlated with humansDeaths was the top performer.

In [23]:
# K-means

pca = PCA(n_components=2)
components = pca.fit_transform(X_hd)

labels = KMeans(n_clusters=2, random_state=123).fit_predict(components)
print(X.shape)
print(len(labels))
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))
X['X_hd_pca_kmeans_clusters'] = labels
group_0 = X[X['X_hd_pca_kmeans_clusters'] == 0]
group_1 = X[X['X_hd_pca_kmeans_clusters'] == 1]

get_pvalue_2clusters(X.columns)

NameError: name 'X_hd' is not defined

## Agglomerative Clustering for all features with PCA Components

In [24]:
def agglomerative_clustering(df, dr_technique, **params):
    clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    dbs = []      
       

    if dr_technique=='PCA':
        pca = PCA(n_components=2)
        components = pca.fit_transform(df)
       
    elif dr_technique=='UMAP': 
        parameters = list(params.keys())
        number_of_neighbors = params[parameters[0]]
        min_distance = params[parameters[1]]
        components = umap.UMAP(n_neighbors=number_of_neighbors, min_dist=min_distance, metric='correlation').fit_transform(df)

    # looping through cluster numbers to find the best cluster
    silhouettes_ward = {} # clusters are keys, silhoutte scores are values
    silhouettes_ward_list = []
    for cluster in clusters:
        agg_cluster = AgglomerativeClustering(linkage='ward',
                                      affinity='euclidean',
                                      n_clusters=cluster)
        labels = agg_cluster.fit_predict(components)
        silhouette = metrics.silhouette_score(components, labels, metric='euclidean')
        silhouettes_ward_list.append(silhouette)
        silhouettes_ward[cluster] = silhouette
        db = metrics.davies_bouldin_score(components, labels)
        dbs.append(db)
        
     
    silhouettes_complete = {} # silhoutte scores are keys, clusters are values
    silhouettes_complete_list = []
    for cluster in clusters:
        agg_cluster = AgglomerativeClustering(linkage='complete',
                                      affinity='euclidean',
                                      n_clusters=cluster)
        labels = agg_cluster.fit_predict(components)
        silhouette = metrics.silhouette_score(components, labels, metric='euclidean')
        silhouettes_complete_list.append(silhouette)
        silhouettes_complete[cluster] = silhouette
        db = metrics.davies_bouldin_score(components, labels)
        dbs.append(db)
       
    silhouettes_average = {} # silhoutte scores are keys, clusters are values
    silhouettes_average_list = []
    for cluster in clusters:
        agg_cluster = AgglomerativeClustering(linkage='average',
                                      affinity='euclidean',
                                      n_clusters=cluster)
        labels = agg_cluster.fit_predict(components)
        silhouette = metrics.silhouette_score(components, labels, metric='euclidean')
        silhouettes_average_list.append(silhouette)
        silhouettes_average[cluster] = silhouette
        db = metrics.davies_bouldin_score(components, labels)
        dbs.append(db)
       
    best_ward_cluster = getKeysByValue(silhouettes_ward, max(silhouettes_ward_list))
    best_ward_cluster = best_ward_cluster[0]
    best_complete_cluster = getKeysByValue(silhouettes_complete, max(silhouettes_complete_list))
    best_complete_cluster = best_complete_cluster[0]
    best_average_cluster = getKeysByValue(silhouettes_average, max(silhouettes_average_list))
    best_average_cluster = best_average_cluster[0]
    # plotting the silhouette scores
    plt.title('Silhouette Scores for Number of Clusters')  
    plt.scatter(list(silhouettes_ward.keys()), list(silhouettes_ward.values()), color='blue')
    plt.scatter(list(silhouettes_complete.keys()), list(silhouettes_complete.values()), color='green')
    plt.scatter(list(silhouettes_average.keys()), list(silhouettes_average.values()), color='red')
    plt.show()

    print('Best Silhouette score with ward linkage is {} with {} clusters. '.format(silhouettes_ward[best_ward_cluster], best_ward_cluster))
    print('Best Silhouette score with complete linkage is {} with {} clusters. '.format(silhouettes_complete[best_complete_cluster], best_complete_cluster))
    print('Best Silhouette score with average linkage is {} with {} clusters. '.format(silhouettes_average[best_average_cluster], best_average_cluster))
    print('Best Davies Bouldin score: ', min(dbs))

## Cluster Labels for humansDeaths features with PCA Components

In [ ]:
# Agglomerative Clustering with humansDeaths features

pca = PCA(n_components=2)
pca_components = pca.fit_transform(X_hd)

agg_cluster = AgglomerativeClustering(linkage='ward', 
                                      affinity='euclidean',
                                      n_clusters=2)
labels = agg_cluster.fit_predict(umap_results)
print(X.shape)
print(len(labels))
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))
X['X_hd_agg_clusters'] = labels
group_0 = X[X['X_hd_agg_clusters'] == 0]
group_1 = X[X['X_hd_agg_clusters'] == 1]

get_pvalue_2clusters(X.columns)

## DBSCAN with PCA Components

In [ ]:
def dbscan_find_clusters(eps, min_samples, X):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X)
    num_labels = len(set(labels))
    num_samples = X.shape[0]
    silhouettes_dbscan = {}
    if num_labels == 1 or num_labels == num_samples:
          silhouette = -1
          silhouettes_dbscan[silhouette] = min_samples
          print('{} min samples and {} eps have no clusters.'.format(min_samples, eps))
    else:
          silhouette = metrics.silhouette_score(X, labels)
          print('{} min samples and {} eps have {} silhouette score with {} clusters.'.format(min_samples, eps, silhouette, num_labels))
          return {
            'eps': eps,
            'min_samples': min_samples,
            'silhouette_score': silhouette,
            'davies_bouldin': metrics.davies_bouldin_score(X, labels)
            }
          silhouettes_dbscan[silhouette] = min_samples
    plt.title('Silhouette Scores for {} eps'.format(eps))
    plt.scatter(list(silhouettes_dbscan.values()), list(silhouettes_dbscan.keys()), color='blue')
    plt.xlabel('Min Samples')
    plt.ylabel('Silhouette Score')

## DBSCAN with All Features with PCA Components¶

In [ ]:
pca = PCA(n_components=2)
pca_components = pca.fit_transform(X_all)

results = [dbscan_find_clusters(eps, min_samples, pca_components) for eps in [0.5, 1, 2] for min_samples in [5,10,15,20]]